# Multi-input and multi-output models using keras

用keas构建一个多输入多输出的模型

see:https://keras.io/getting-started/functional-api-guide/#_3

参考中文版官方文档：https://keras.io/zh/getting-started/functional-api-guide/#_3

有时候我们的输入不仅仅是文字，还需要有图片辅助或者语音辅助，这时我们就可以用函数式API来构建这么一个网络

The whole framework for processing text and image

In [2]:
import keras
from keras.layers import Input, Embedding, LSTM, Dense, Dropout
from keras.models import Model

Using TensorFlow backend.


In [3]:
#图像部分简单定义一个网络
img_maxlen = 100
img_input = Input(shape=(img_maxlen,))
x1 = Dense(1000, activation='relu')(img_input)
x1 = Dropout(0.5)(x1)
x1 = Dense(600, activation='relu')(x1)
img_output = Dense(10)(x1)

In [4]:
#文字部分用LSTM
text_maxlen = 100
text_input = Input(shape=(text_maxlen,),dtype='int32')
x2 = Embedding(output_dim=512, input_dim=10000, input_length=text_maxlen)(text_input)
x2 = LSTM(32)(x2)
x2 = Dropout(0.5)(x2)
text_output = Dense(10)(x2)

In [5]:
#将图像处理的输出与 LSTM 层的输出连接起来，输入到模型中
x = keras.layers.concatenate([img_output, text_output])

# 堆叠多个全连接网络层
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# 最后添加主要的逻辑回归层
output = Dense(3, activation='softmax', name='output')(x)

#定义一个具有两个输入和一个输出的模型：
model = Model(inputs=[img_input, text_input], outputs=[output])

In [6]:
#模型编译
from keras.optimizers import Adam

model.compile(
    optimizer=Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
             )